In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from pycocotools.coco import COCO
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (20.0, 20.0)

In [2]:
def sec2frame(seconds, fps):
    return round(seconds*fps)

def get_vid_meta(filename):
    video = cv2.VideoCapture(filename)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps
    return fps, duration, frame_count


def process_single(annot_fn, fps, fout, img_folder):
    with open(annot_fn) as f:
        contents = json.load(f)

    annot = contents['metadata']
    attr = contents['attribute']

    # not doing anything with the temporal 
    # section....... 
    obj_seg = []
    for d in annot.values():
        if len(d['z']) == 1:
            obj_seg.append(d)
    # just using object present & object id for this....
    name2i_attr = {d['aname']: i for i, d in attr.items()}
    object_present = name2i_attr['object_present']
    object_id = name2i_attr['object_id']
    object_label = name2i_attr['object_label']

    # coco starting out  
    coco_dict = { 'annotations': [ ]}

    # just gonna use object present & object id
    cat2id = {}
    id_count = 0
    dropped_items = []
    for i, obj in enumerate(obj_seg):
        av = obj['av']
        # ==2 checks for rectangle
        if (object_label not in av  and object_present not in av ) or obj['xy'][0] != 2:
            if len(dropped_items) == 0:
                dropped_items.append(name2i_attr)
            dropped_items.append(obj)
            continue 
        if av.get(object_label, None):
            cat = av[object_label].strip()
        else:
            cat = av[object_present].strip()

        c_id = av.get(object_id, '0').strip()
        # adding to categories.....if needed
        if (cat, c_id) not in cat2id:
                cat2id[(cat, c_id)] = id_count
                id_count +=1

        coco_dict['annotations'].append({
                'id': i,
                # we need keypoint or segmentation for some reason....
                'keypoints': None,
                'category_id': cat2id[(cat, c_id)],
                "image_id": sec2frame(obj['z'][0], fps),
                'bbox': obj['xy'][1:],
                'area': obj['xy'][-1]*obj['xy'][-2]

        })
            
            
            
    imgs = sorted(os.listdir(img_folder))
    paths = [f'{img_folder}/{f}' for f in imgs]
    assert(int(paths[-1].split('.jpg')[0].split('-')[-1]) == len(paths))
    coco_dict['images'] = [{'id':i, 'file_name':fn} for i, fn in enumerate(paths)]
    coco_dict['categories'] = [{'id': i, 'name': f'{cat}_{im}', 'supercategory':cat} 
                               for (cat, im), i in cat2id.items()]
    with open(fout, 'w') as f:
        json.dump(coco_dict, f)
    
    return dropped_items



In [3]:
frames = 'shark_frames'

prefixes = [f for f in os.listdir(frames) if os.path.isdir(f'{frames}/{f}')]

In [12]:
prefixes

['20210910_SUPInteraction',
 '20210403_SharkAndBodyboarders',
 '20210910_SurferTowardsShark',
 '20210401_ThreeSharksVeryCloseSurfers',
 '2020731_JWSSeaLIonSUP',
 '20210812_SharkOutsideLineUP',
 '20210121_SharkNearTwoSurfers',
 '20210121_SUPNexttoShark',
 '20210615_SharkUnderSurfer',
 '20210812_JamesWithShark2',
 '20210401_ThreeSurfersTwoSharks',
 '20200805_OneSharkSUPFollowsKids',
 '20210910_SharkInsideLineUp_GreatInteraction',
 '20210812_JamesWithShark3',
 '20210910_SharkSwimmerSurfer',
 'DJI_0386',
 '20210812_JamesElizabethBrianWithShark',
 '20210812_JamesWithShark',
 '20210910_SharkSpeedingPastSurfers',
 '20200805_OneSharkSUPSurfers',
 '20210403_SharkApproachesSurfer']

In [28]:
dropped_things = {}
d_folder ='data'
annot_folder = 'coco_files'
new_prefixes = []
for p in prefixes:
    new_p = f'{d_folder}/{p}'
    if os.path.isfile(new_p + '.json') and os.path.isfile(new_p + '.mp4'):
        print(new_p)
        fps, _, _ = get_vid_meta(new_p + '.mp4')
        fout = f'{annot_folder}/{p}_coco.json'
        dropped_items = process_single(new_p+'.json', fps, fout, f'{frames}/{p}')
        if len(dropped_items) > 0:
            print(len(dropped_items))
            dropped_things[p] = dropped_items
        else:
            new_prefixes.append(p)

data/20210910_SUPInteraction
data/20210403_SharkAndBodyboarders
2
data/20210910_SurferTowardsShark
data/20210401_ThreeSharksVeryCloseSurfers
data/2020731_JWSSeaLIonSUP
data/20210812_SharkOutsideLineUP
data/20210121_SharkNearTwoSurfers
data/20210121_SUPNexttoShark
7
data/20210615_SharkUnderSurfer
data/20210812_JamesWithShark2
data/20210401_ThreeSurfersTwoSharks
data/20200805_OneSharkSUPFollowsKids
data/20210910_SharkInsideLineUp_GreatInteraction
2
data/20210812_JamesWithShark3
data/20210910_SharkSwimmerSurfer
data/DJI_0386
data/20210812_JamesElizabethBrianWithShark
321
data/20210812_JamesWithShark
data/20200805_OneSharkSUPSurfers


In [29]:
new_prefixes

['20210910_SUPInteraction',
 '20210910_SurferTowardsShark',
 '20210401_ThreeSharksVeryCloseSurfers',
 '2020731_JWSSeaLIonSUP',
 '20210812_SharkOutsideLineUP',
 '20210121_SharkNearTwoSurfers',
 '20210615_SharkUnderSurfer',
 '20210812_JamesWithShark2',
 '20210401_ThreeSurfersTwoSharks',
 '20200805_OneSharkSUPFollowsKids',
 '20210812_JamesWithShark3',
 '20210910_SharkSwimmerSurfer',
 'DJI_0386',
 '20210812_JamesWithShark',
 '20200805_OneSharkSUPSurfers']

In [70]:
cat coco_files/20200805_OneSharkSUPFollowsKids_coco.json

{"annotations": [{"id": 0, "keypoints": null, "category_id": 0, "image_id": 0, "bbox": [1626.585, 148.171, 69.147, 102.073], "area": 7058.041731}, {"id": 1, "keypoints": null, "category_id": 1, "image_id": 1, "bbox": [2011.829, 701.341, 62.561, 121.83], "area": 7621.80663}, {"id": 2, "keypoints": null, "category_id": 0, "image_id": 3, "bbox": [1626.585, 144.878, 69.147, 98.781], "area": 6830.409807000001}, {"id": 3, "keypoints": null, "category_id": 0, "image_id": 6, "bbox": [1620, 144.878, 79.024, 105.366], "area": 8326.442784}, {"id": 4, "keypoints": null, "category_id": 1, "image_id": 4, "bbox": [2018.415, 704.634, 52.683, 115.244], "area": 6071.399652}, {"id": 5, "keypoints": null, "category_id": 0, "image_id": 9, "bbox": [1620, 135, 79.024, 111.951], "area": 8846.815824}, {"id": 6, "keypoints": null, "category_id": 0, "image_id": 12, "bbox": [1616.707, 131.707, 92.195, 118.537], "area": 10928.518715}, {"id": 7, "keypoints": null, "category_id": 0, "image_id": 15, "bbox": [1626.585

FollowsKids-1026.jpg"}, {"id": 1026, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1027.jpg"}, {"id": 1027, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1028.jpg"}, {"id": 1028, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1029.jpg"}, {"id": 1029, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1030.jpg"}, {"id": 1030, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1031.jpg"}, {"id": 1031, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1032.jpg"}, {"id": 1032, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1033.jpg"}, {"id": 1033, "file_name": "shark_frames/20200805_OneSharkSUPFollowsKids/20200805_OneSharkSUPFollowsKids-1034.jpg"}, {"id": 1034, "file_name": "shark_frames/20200805

In [71]:
annFile = 'coco_files/20200805_OneSharkSUPFollowsKids_coco.json'
coco=COCO(annFile)

# annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
# anns = coco.loadAnns(annIds)
# coco.showAnns(anns

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [72]:
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))


COCO categories: 
shark_0 1_0 human_0 1_00

COCO supercategories: 
1 shark human


In [73]:
annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
# prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
# print 'Running demo for *%s* results.'%(annType)


In [74]:
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [75]:
coco.dataset['categories']

[{'id': 0, 'name': 'shark_0', 'supercategory': 'shark'},
 {'id': 1, 'name': '1_0', 'supercategory': '1'},
 {'id': 2, 'name': 'human_0', 'supercategory': 'human'},
 {'id': 3, 'name': '1_00', 'supercategory': '1'}]

In [64]:
def generate_labeled_images(annFile, img_folder, box_thickness=10, output_folder='test', draw_text=True, 
                            fontScale = 4, color = (36,255,12), # neon green
                            font = cv2.FONT_HERSHEY_SIMPLEX, text_thickness = 10):
    # getting images & frames
    imgs = sorted(os.listdir(img_folder))
    paths = [f'{img_folder}/{f}' for f in imgs]
    assert(int(paths[-1].split('.jpg')[0].split('-')[-1]) == len(paths))
    frame_num = len(paths)
    
    # getting annotations 
    coco=COCO(annFile)
    cats = coco.loadCats(coco.getCatIds())
    nms=[cat['name'] for cat in cats]
    catIds = coco.getCatIds(catNms=nms);
    
    id2catname = {d['id']: d['name'] for d in coco.dataset['categories']}

    # some defaults 
    for i in range(frame_num):
        fn = paths[i]
        image = cv2.cvtColor(cv2.imread(fn), cv2.COLOR_BGR2RGB)
        annIds = coco.getAnnIds(imgIds=i, catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        # adding annotations 
        for v in anns:
            rects = np.array(v['bbox']).astype(int)
            s = rects[:2]
            e = rects[:2] + rects[2:]
            image = cv2.rectangle(image, s, e, color, box_thickness)
            # drawing label id 
            if draw_text: 
                text = id2catname[v['category_id']]
                image = cv2.putText(image, text, s - np.array([0,10]),font,
                                    fontScale, color, text_thickness)
        if draw_text and i < 100:
            org = (50,150)
            text = img_folder.split('/')[-1]
            image = cv2.putText(image, text, org, font, fontScale, color, text_thickness)
        
        out_fn = f'{output_folder}/{imgs[i]}'
        if len(anns) >0 and not cv2.imwrite(out_fn, image):
            print(out_fn)

In [65]:
# where we ended 
prev_run = '20210812_JamesWithShark2'
convert_imgs = new_prefixes
for pre in convert_imgs:
    annot_fn = f'{annot_folder}/{pre}_coco.json'
    frames_fn = f'{frames}/{pre}'
    print(pre)
    generate_labeled_images(annot_fn, frames_fn)
    

20210910_SUPInteraction
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
20210910_SurferTowardsShark
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
20210401_ThreeSharksVeryCloseSurfers
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2020731_JWSSeaLIonSUP
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
20210812_SharkOutsideLineUP
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
20210121_SharkNearTwoSurfers
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
20210615_SharkUnderSurfer
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
20210812_JamesWithShark2
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
20210401_ThreeSurfersTwoSharks
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
202

In [ ]:
convert_imgs

In [11]:
ls coco_files/

20200805_OneSharkSUPFollowsKids_coco.json
20200805_OneSharkSUPSurfers_coco.json
2020731_JWSSeaLIonSUP_coco.json
20210121_SUPNexttoShark_coco.json
20210121_SharkNearTwoSurfers_coco.json
20210401_ThreeSharksVeryCloseSurfers_coco.json
20210401_ThreeSurfersTwoSharks_coco.json
20210403_SharkAndBodyboarders_coco.json
20210615_SharkUnderSurfer_coco.json
20210812_JamesElizabethBrianWithShark_coco.json
20210812_JamesWithShark2_coco.json
20210812_JamesWithShark3_coco.json
20210812_JamesWithShark_coco.json
20210812_SharkOutsideLineUP_coco.json
20210910_SUPInteraction_coco.json
20210910_SharkInsideLineUp_GreatInteraction_coco.json
20210910_SharkSwimmerSurfer_coco.json
20210910_SurferTowardsShark_coco.json
DJI_0386_coco.json
